# Suspicious Userinit Child Process
Detects a suspicious child process of userinit

## Rule Content
```
- title: Suspicious Userinit Child Process
  id: b655a06a-31c0-477a-95c2-3726b83d649d
  status: experimental
  description: Detects a suspicious child process of userinit
  references:
  - https://twitter.com/SBousseaden/status/1139811587760562176
  author: Florian Roth (rule), Samir Bousseaden (idea)
  date: 2019/06/17
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentImage: '*\userinit.exe'
    filter1:
      CommandLine: '*\\netlogon\\*'
    filter2:
      Image: '*\explorer.exe'
    condition: selection and not filter1 and not filter2
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Administrative scripts
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_parent_path.keyword:*\\userinit.exe AND (NOT (process_command_line.keyword:*\\netlogon\\*))) AND (NOT (process_path.keyword:*\\explorer.exe)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()